![AIRBNB](https://www.stevenridercpa.au/wp-content/uploads/2022/09/airbnb-tax.jpeg)

# Airbnb - Price Prediction
-------

## Peroblem Statement

A dataset containing information of accommodations published in AirBnB with their respective prices is presented. The size of the train dataset is approximately 1.5 Gb, and 0.5 Gb for the test dataset. This has 84 predictor variables that can be used as they see fit.

The objective is to assign the correct price to the listed accommodations. 

In addition to the dataset, you are provided with this notebook containing the data loading script and a baseline model corresponding to a feed forward architecture.

------

## Guidelines

### Participation in Kaggle Competition

The goal of this section is to participate in the Kaggle competition and achieve a Mean Absolute Error of less than 70 points. [->Link to the competition<-](https://www.kaggle.com/t/69c648e3aa214d1f812bf2314c8d4ffa).

### Use of Grid Search (or equivalent)

To meet the requirement for optimal model searching, a comprehensive and methodical grid search must be conducted. We strongly recommend [Weights and Biases](https://wandb.ai/site).

### You must also research and implement the following techniques

- [Batch Normalization](https://machinelearningmastery.com/how-to-accelerate-learning-of-deep-neural-networks-with-batch-normalization/)
- [Gradient Normalization and/or Gradient Clipping](https://machinelearningmastery.com/how-to-avoid-exploding-gradients-in-neural-networks-with-gradient-clipping/)


-------

## Setup
### Imports

In [2]:
import pandas as pd
import numpy as np
# import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

### Seeds

In [ ]:
np.random.seed(117)
# tf.random.set_seed(117)

## 2. Carga de datos

In [3]:
TRAIN_PATH = './data/public_train_data.csv'
df = pd.read_csv(TRAIN_PATH)

##  3. Análisis exploratorio de datos
### 3.1 Dimensiones

In [4]:
df.shape

(326287, 85)

### Obtain information about columns and data types

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326287 entries, 0 to 326286
Data columns (total 85 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              326287 non-null  int64  
 1   Last Scraped                    326286 non-null  object 
 2   Name                            326018 non-null  object 
 3   Summary                         315651 non-null  object 
 4   Space                           228792 non-null  object 
 5   Description                     326188 non-null  object 
 6   Experiences Offered             326287 non-null  object 
 7   Neighborhood Overview           192513 non-null  object 
 8   Notes                           130729 non-null  object 
 9   Transit                         200649 non-null  object 
 10  Access                          177108 non-null  object 
 11  Interaction                     169193 non-null  object 
 12  House Rules     

In [19]:
# Select only numeric columns
df_numeric = df.select_dtypes(include=['float64', 'int64'])

# Calculate the correlation matrix
correlation_matrix = df_numeric.corr()

# Get correlation of all features with 'price'
price_correlation = correlation_matrix['Price'].sort_values(ascending=False)

# Filter out the features with a correlation above a certain threshold, for example 0.3
important_features = price_correlation[abs(price_correlation) >= 0.1]

# Room Type, Smart Location	

In [20]:
print(important_features)

Price               1.000000
Cleaning Fee        0.739040
Security Deposit    0.394896
Accommodates        0.373885
Bedrooms            0.361354
Beds                0.293338
Extra People        0.274710
Bathrooms           0.260185
Guests Included     0.192998
Square Feet         0.111586
Name: Price, dtype: float64


### 3.3 Visualizar las primeras filas del dataset

In [14]:
pd.set_option('display.max_columns', None)

df.head(10)

,id,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,Notes,Transit,Access,Interaction,House Rules,Thumbnail Url,Medium Url,Picture Url,XL Picture Url,Host ID,Host URL,Host Name,Host Since,Host Location,Host About,Host Response Time,Host Response Rate,Host Acceptance Rate,Host Thumbnail Url,Host Picture Url,Host Neighbourhood,Host Listings Count,Host Total Listings Count,Host Verifications,Street,Neighbourhood,Neighbourhood Cleansed,Neighbourhood Group Cleansed,City,State,Zipcode,Market,Smart Location,Country Code,Country,Latitude,Longitude,Property Type,Room Type,Accommodates,Bathrooms,Bedrooms,Beds,Bed Type,Amenities,Square Feet,Security Deposit,Cleaning Fee,Guests Included,Extra People,Minimum Nights,Maximum Nights,Calendar Updated,Has Availability,Availability 30,Availability 60,Availability 90,Availability 365,Calendar last Scraped,Number of Reviews,First Review,Last Review,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features,Price
0,0,2017-05-12,Grand Loft in the heart of historic Antwerp,Best location for visiting Antwerp!! Beautiful...,Welcome in Antwerp!! The loft is situated on t...,Best location for visiting Antwerp!! Beautiful...,none,NaN,NaN,NaN,The Loft will be only accessible for you and y...,Please feel free to ask me anything and I will...,- It would be greatly appreciated to be respec...,https://a0.muscache.com/im/pictures/ddd94a2b-b...,https://a0.muscache.com/im/pictures/ddd94a2b-b...,https://public.opendatasoft.com/api/explore/v2...,https://a0.muscache.com/im/pictures/ddd94a2b-b...,50969699,https://www.airbnb.com/users/show/50969699,Petra,2015-12-10,"Antwerp, Flanders, Belgium","Hi, I'm Petra , a Belgian interior designer. ...",within an hour,100.0,NaN,https://a0.muscache.com/im/pictures/5271a897-2...,https://a0.muscache.com/im/pictures/5271a897-2...,NaN,2.0,2.0,"email,phone,reviews","Antwerp, Flanders, Belgium",NaN,Historisch Centrum,NaN,Antwerp,Flanders,NaN,Antwerp,"Antwerp, Belgium",BE,Belgium,51.219388,4.403444,Apartment,Entire home/apt,6.0,1.0,2.0,4.0,Real Bed,"TV,Cable TV,Wireless Internet,Kitchen,Elevator...",NaN,200.0,50.0,6.0,25.0,2.0,51.0,6 weeks ago,NaN,15.0,36.0,54.0,317.0,2017-05-12,9.0,2017-01-29,2017-04-26,98.0,9.0,9.0,9.0,9.0,10.0,9.0,NaN,NaN,strict,2.0,2.60,"51.21938762207894, 4.4034442505151885","Host Has Profile Pic,Instant Bookable",159.0
1,1,2017-05-03,"CHARMING, CLEAN & COZY BUNGALOW!",Very centrally located and less than 15 min fr...,"Well lit, private entrance with small patio.",Very centrally located and less than 15 min fr...,none,"Quiet. Pretty tree lined streets, safe area.",Has dining table and high back desk chair.,"Uber, bus line and metro link is less than 5 m...",Front parking,NaN,"No smoking in unit, quiet hours after 10pm, no...",https://a0.muscache.com/im/pictures/f025506b-d...,https://a0.muscache.com/im/pictures/f025506b-d...,https://public.opendatasoft.com/api/explore/v2...,https://a0.muscache.com/im/pictures/f025506b-d...,50132213,https://www.airbnb.com/users/show/50132213,Laurie,2015-11-29,"Los Angeles, California, United States",NaN,within a few hours,100.0,NaN,https://a0.muscache.com/im/pictures/58bde558-c...,https://a0.muscache.com/im/pictures/58bde558-c...,Valley Glen,1.0,1.0,phone,"Valley Glen, Los Angeles, CA 91401, United States",Valley Glen,Valley Glen,NaN,Los Angeles,CA,91401,Los Angeles,"Los Angeles, CA",US,United States,34.189269,-118.419935,House,Private room,2.0,1.0,1.0,1.0,Real Bed,"TV,Internet,Wireless Internet,Air conditioning...",NaN,100.0,50.0,1.0,0.0,1.0,7.0,4 weeks ago,NaN,29.0,59.0,89.0,89.0,2017-05-02,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"City of Los Angeles, CA",flexible,1.0,NaN,"34.1892692286356, -118.41993491931177","Host Has Profile Pic,Is Location Exact",49.0
2,2,2017-05-09,la

### 3.4 Estadísticas descriptivas

In [ ]:
df.describe()

In [ ]:
df.columns

## 4. Modelo Baseline

### 4.1 Seleccionar características relevantes

In [ ]:
features = ['Bathrooms', 'Bedrooms']  # Reemplaza con las características relevantes
target = 'Price' 
df = df[[*features, target]]
df.dropna(inplace=True)

In [ ]:
X = df[features]
y = df[target]

### 4.2 Dividir los datos en conjuntos de entrenamiento y prueba

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 4.3 Definir el modelo

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense


model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1, activation='relu')  # Capa de salida para la predicción del precio
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

### 4.4 Entrenar

In [ ]:
history = model.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.2)

### 4.5 Evaluar en Test

In [ ]:
loss, mae = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {mae}')

## 5 Generación de salida para competencia en Kaggle

In [ ]:
file_path2 = './airbnb_data/private_data_to_predict.csv'
data_for_kaggle = pd.read_csv(file_path2)

In [ ]:
kaggle_results = model.predict(data_for_kaggle[features])
test_ids = data_for_kaggle['id']
test_ids = np.array(test_ids).reshape(-1,1)
output = np.stack((test_ids, kaggle_results), axis=-1)
output = output.reshape([-1, 2])
df = pd.DataFrame(output)
df.columns = ['id','expected']  
df['expected'] = df['expected'].fillna(0)   
df.to_csv("output_to_submit.csv", index = False, index_label = False)


## 6 Ejemplo de uso de Weights and Biases

In [ ]:
from tensorflow.keras.layers import Dropout

def get_model(neurons, optimizer, dropout):
    layers = []
    input_shape = (X_train.shape[1],)
    for n in neurons:
        layers.append(Dense(n, activation = "relu", input_shape = input_shape))
        layers.append(Dropout(dropout))
        input_shape = (n,)
        
    model = Sequential(layers)
    model.compile(optimizer = optimizer, loss='mean_squared_error', metrics=['mae'])
    return model

In [ ]:
# Import the W&B Python Library and log into W&B
import wandb

wandb.login()

#Creamos un proyecto en WandB a través de su interfaz
project = "obligatorio_dl"
entity = "franzmayr"

In [ ]:
import traceback

def run_train():
    try:       
        with wandb.init(config = None, project = project, entity=entity):     
            # initialize model
            config = wandb.config
            print(config)
            model= get_model(config.neurons, config.optimizer, config.dropout)
            tf.keras.backend.clear_session()
            wandb_callback = wandb.keras.WandbCallback()
            model.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.2, callbacks=[wandb_callback], max_queue_size=3, workers=2)

    except Exception as e:
        # exit gracefully, so wandb logs the problem
        print(traceback.print_exc(), file=sys.stderr)
        exit(1)

In [ ]:
import pprint

sweep_config = {
'name': 'sweep_example',
'method': 'grid',
'metric': {
    'name': 'val_loss',
    'goal': 'minimize'   
},
'parameters': {
    'dropout':{'value': 0.1},
    'neurons':{
        'values': [[32,2],[64,32,2]]
        },
    'optimizer': {
        'values': ['adam', 'sgd']
        }
}
}

pprint.pprint(sweep_config)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project = project, entity = entity)

In [ ]:
wandb.agent(sweep_id, function = run_train, count=10, project = project, entity = entity)